In [1]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

legislatura = 'Legislatura XLVIII'

legisladores_df = pd.read_csv('data/%s/legisladores.csv' % legislatura)
legisladores_df.head()

,cuerpo,email,id_legislador,lema,nombre
0,NaN,eagazzi@parlamento.gub.uy,5146,NaN,"Agazzi, Ernesto"
1,NaN,NaN,2743,NaN,"Alcorta, Ricardo"
2,NaN,secretaria.ministro@mrree.gub.uy,8883,NaN,"Almagro Lemes, Luis Leonardo"
3,SENADO,valonso@parlamento.gub.uy,9891,PARTIDO NACIONAL,"Alonso, Verónica"
4,NaN,nolivera@parlamento.gub.uy,11535,NaN,"Álvarez López, María Dolores"


## Dimensiones

### Asistencia
Presiencia en el plenario y comisiones<br>
Promedio de la presencia en el plenario y las comisiones (asistencias / citaciones)

In [2]:
asistencia_plenario_df = pd.read_csv('data/%s/asistencia_plenario.csv' % legislatura)
asistencia_comision_df = pd.read_csv('data/%s/asistencia_comisiones.csv' % legislatura)
asistencia_df = pd.concat([asistencia_plenario_df[['id_legislador', 'asistencias', 'citaciones']], asistencia_comision_df[['id_legislador', 'asistencias', 'citaciones']]])
asistencia_df = asistencia_df.groupby('id_legislador').sum()

# Cálculo de z-score de asistencias
asistencia_df.loc[:,'asistencia_med'] = asistencia_df.apply(lambda x : x['asistencias'] * 1.0 / x['citaciones'], axis=1)
asistencia_media = asistencia_df['asistencia_med'].mean()
asistencia_std = asistencia_df['asistencia_med'].std()
asistencia_df.loc[:,'asistencia_zidx'] = asistencia_df.apply(lambda x : (x['asistencia_med'] * 1.0 - asistencia_media) / asistencia_std, axis=1)
asistencia_df.reset_index(inplace=True)
asistencia_df = asistencia_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
asistencia_df.head()

,id_legislador,asistencias,citaciones,asistencia_med,asistencia_zidx,nombre,cuerpo,lema
0,149,414,454,0.911894,0.224948,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,376,492,0.764228,-0.652918,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,558,665,0.839098,-0.207821,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,359,463,0.775378,-0.586630,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,364,446,0.816143,-0.344282,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Proyectos
Incluye las siguientes actividades:
* Presentación o firma de proyectos de ley
* Proyectos de resolución
* Proyectos de minutas
* Proyectos de declaración de cuerpo
<br><br>El índice se basa en la suma de todos los proyectos presentados

In [3]:
proyectos_presentados_df = pd.read_csv('data/%s/proyectos_presentados.csv' % legislatura)
proyectos_df = pd.read_csv('data/%s/proyectos.csv' % legislatura)
proyectos_presentados_df = proyectos_presentados_df.merge(proyectos_df, on='id_proyecto', how='left')

proyectos_presentados_df = proyectos_presentados_df[proyectos_presentados_df['tipo'].isin(['PROYECTO DE DECLARACION', 'PROYECTO DE LEY', 'PROYECTO DE MINUTA DE COMUNICACION','PROYECTO DE RESOLUCION'])]
proyectos_presentados_df = proyectos_presentados_df[['id_legislador', 'id_proyecto']].groupby('id_legislador').size().reset_index(name='proyectos_total')

proyectos_presentados_media = proyectos_presentados_df['proyectos_total'].mean()
proyectos_presentados_std = proyectos_presentados_df['proyectos_total'].std()
proyectos_presentados_df.loc[:,'proyectos_presentados_zidx'] = proyectos_presentados_df.apply(lambda x : (x['proyectos_total'] * 1.0 - proyectos_presentados_media) / proyectos_presentados_std, axis=1)
proyectos_presentados_df = proyectos_presentados_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
proyectos_presentados_df.head()

,id_legislador,proyectos_total,proyectos_presentados_zidx,nombre,cuerpo,lema
0,149,23,1.839252,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,19,1.331666,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,24,1.966149,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,28,2.473734,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,14,0.697184,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Informante en comisión
Informantes de proyectos en las comisiones en las que participa<br>
El índice se basa en la suma de número de acciones

In [4]:
actuacion_parlamentaria_df = pd.read_csv('data/%s/actuacion_parlamentaria.csv' % legislatura, index_col=0)
informante_comision_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INFORMA']
informante_comision_df = informante_comision_df.groupby('id_legislador').size().reset_index(name='informes_total')
informante_comision_media = informante_comision_df['informes_total'].mean()
informante_comision_std = informante_comision_df['informes_total'].std()
informante_comision_df.loc[:,'informe_comision_zidx'] = informante_comision_df.apply(lambda x : (x['informes_total'] * 1.0 - informante_comision_media) / informante_comision_std, axis=1)
informante_comision_df = informante_comision_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
informante_comision_df.head()


,id_legislador,informes_total,informe_comision_zidx,nombre,cuerpo,lema
0,149,10,0.136277,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,1,-0.571700,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,27,1.473569,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,9,0.057613,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,13,0.372270,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Debate
Interviene en debates sobre proyectos<br>
El índice se basa en la suma de las intervenciones en el plenario

In [5]:
interviene_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INTERVIENE']
interviene_df = interviene_df.groupby('id_legislador').size().reset_index(name='interveciones_total')
interviene_media = interviene_df['interveciones_total'].mean()
interviene_std = interviene_df['interveciones_total'].std()
interviene_df.loc[:,'intervenciones_zidx'] = interviene_df.apply(lambda x : (x['interveciones_total'] * 1.0 - interviene_media) / interviene_std, axis=1)
interviene_df = interviene_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
interviene_df.head()


,id_legislador,interveciones_total,intervenciones_zidx,nombre,cuerpo,lema
0,149,296,3.332254,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,132,1.165414,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,325,3.715415,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,230,2.460233,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,117,0.967227,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Exposiciones
Escribe notas o realiza exposiciones sobre temas determinados.<br>
El índice se basa en la suma de las exposiciones realizadas

In [6]:
expone_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'EXPONE']
expone_df = expone_df.groupby('id_legislador').size().reset_index(name='exposiciones_total')
expone_media = expone_df['exposiciones_total'].mean()
expone_std = expone_df['exposiciones_total'].std()
expone_df.loc[:,'exposiciones_zidx'] = expone_df.apply(lambda x : (x['exposiciones_total'] * 1.0 - expone_media) / expone_std, axis=1)
expone_df = expone_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
expone_df.head()


,id_legislador,exposiciones_total,exposiciones_zidx,nombre,cuerpo,lema
0,149,25,0.650913,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,24,0.599328,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,23,0.547743,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,5,-0.380794,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,19,0.341401,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Pedidos de informe
Solicita un pedido de informes al gobierno.<br>
El índice se basa en la suma de pedidos de informe

In [7]:
pedidos_de_informe_df = pd.read_csv('data/%s/pedidos_informe.csv' % legislatura, index_col=0)
pedidos_de_informe_df = pedidos_de_informe_df[['id_legislador', 'id_proyecto']].groupby('id_legislador').size().reset_index(name='pedidos_informe_total')
pedidos_de_informe_media = pedidos_de_informe_df['pedidos_informe_total'].mean()
pedidos_de_informe_std = pedidos_de_informe_df['pedidos_informe_total'].std()
pedidos_de_informe_df.loc[:,'pedidos_informe_zidx'] = pedidos_de_informe_df.apply(lambda x : (x['pedidos_informe_total'] * 1.0 - pedidos_de_informe_media) / pedidos_de_informe_std, axis=1)
pedidos_de_informe_df = pedidos_de_informe_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
pedidos_de_informe_df.head()


,id_legislador,pedidos_informe_total,pedidos_informe_zidx,nombre,cuerpo,lema
0,149,34,1.314276,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,14,0.017485,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,7,-0.436392,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,14,0.017485,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,2,-0.760590,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


In [8]:
indice_senadores_df = asistencia_df[['nombre', 'cuerpo', 'lema', 'id_legislador', 'asistencia_zidx']].merge(proyectos_presentados_df[['id_legislador', 'proyectos_presentados_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(informante_comision_df[['id_legislador', 'informe_comision_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(interviene_df[['id_legislador', 'intervenciones_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(expone_df[['id_legislador', 'exposiciones_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(pedidos_de_informe_df[['id_legislador', 'pedidos_informe_zidx']], on='id_legislador')
indice_senadores_df.loc[:, 'zidx'] = indice_senadores_df['asistencia_zidx'] +indice_senadores_df['asistencia_zidx'] + indice_senadores_df['proyectos_presentados_zidx'] +indice_senadores_df['informe_comision_zidx'] + indice_senadores_df['intervenciones_zidx'] + indice_senadores_df['exposiciones_zidx'] + indice_senadores_df['pedidos_informe_zidx']
indice_senadores_df.sort_values('zidx', ascending=False).head()


,nombre,cuerpo,lema,id_legislador,asistencia_zidx,proyectos_presentados_zidx,informe_comision_zidx,intervenciones_zidx,exposiciones_zidx,pedidos_informe_zidx,zidx
12,"Bordaberry Herrán, Juan Pedro",SENADO,PARTIDO COLORADO,922,0.162118,1.712356,1.866890,7.599872,1.888962,2.157191,15.549506
7,"Martínez Huelmo, Rubén",SENADO,PARTIDO FRENTE AMPLIO,408,0.492223,0.950977,7.137391,2.658420,0.599328,-0.760590,11.569972
21,"Abdala, Pablo",REPRESENTANTE,PARTIDO NACIONAL,3062,-0.010196,2.854424,0.136277,4.917747,1.940547,1.703314,11.531917
63,"Arocena Argul, José Andrés",REPRESENTANTE,PARTIDO NACIONAL,9953,-1.174155,2.854424,-0.178379,0.835103,7.872863,1.638474,10.674175
5,"Mieres, Pablo",SENADO,PARTIDO INDEPENDIENTE,327,0.009171,1.712356,0.529598,4.587436,0.805669,1.832993,9.486393


In [9]:
actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'].isnull()].head()

,fecha,id,id_legislador,tipo
detalle,,,,
Renuncia por opción el 15/02/2015,2015-02-15,2,9891,NaN
Presenta el proyecto de ley. Carpeta: CSS/172/2015; PACIENTES CON TRATAMIENTO MEDICO EN EL EXTERIOR. TRASLADO. MILLAS PASAJES OFICIALES. USO. Se da cuenta al cuerpo: tomo 534 pág. 126 d.s. 7,2015-03-24,10,9891,NaN
"Presenta el proyecto de minuta de comunicación. Carpeta: CSS/225/2015; CIENCIA, TECNOLOGÍA E INNOVACIÓN. MINISTERIO. CREACIÓN. Se da cuenta al cuerpo: tomo 535 pág. 4 d.s. 12",2015-05-11,14,9891,NaN
"Presenta el proyecto de ley. Carpeta: CSS/247/2015; ORIBE, Josefa. ESCUELA Nº 183 (MONTEVIDEO). DENOMINACIÓN. Se da cuenta al cuerpo: tomo 535 pág. 251 d.s. 17",2015-06-02,15,9891,NaN
"Presenta el proyecto de resolución. Carpeta: CSS/300/2015; SINGLET, Manuel. FALLECIMIENTO. HOMENAJE.",2015-07-14,18,9891,NaN


In [10]:
df = pd.read_csv('data/%s/convocatoria.csv' % legislatura, infer_datetime_format=True, parse_dates=['fecha_ini', 'fecha_fin'])
convocatoria_df = df.merge(legisladores_df[['id_legislador', 'nombre']], on='id_legislador', how='left')
convocatoria_df.head()

,camara,departamento,duracion_segundos,fecha_fin,fecha_ini,id,id_legislador,lema,sublema,titular,nombre
0,Cámara de Senadores,NaN,NaN,NaT,2015-02-15,0,9891,PARTIDO NACIONAL,NaN,NaN,"Alonso, Verónica"
1,Cámara de Representantes,MONTEVIDEO,60.0,2015-02-15 00:01:00,2015-02-15,1,9891,NaN,2014,NaN,"Alonso, Verónica"
2,Cámara de Senadores,NaN,0.0,2016-12-14 00:00:00,2015-02-15,2,5146,NaN,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,NaN,"Agazzi, Ernesto"
3,Cámara de Senadores,NaN,86340.0,2015-03-18 23:59:00,2015-03-18,3,2743,NaN,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,NaN,"Alcorta, Ricardo"
4,Cámara de Senadores,NaN,86340.0,2015-04-08 23:59:00,2015-04-07,4,2743,PARTIDO FRENTE AMPLIO,MAS FRENTE AMPLIO PARA MAS DESARROLLO CON IGUA...,"Ayala, Patricia","Alcorta, Ricardo"


In [11]:
convocatoria_df[['nombre', 'duracion_segundos']].groupby('nombre').sum()

,duracion_segundos
nombre,
"Abdala, Pablo",0.0
"Abreu Castillo, Aníbal",662040.0
"Acosta, Auro",5439420.0
"Acosta, José Luis",1640460.0
"Agazzi, Ernesto",474900.0
"Agüero Demetrio, Nelson Leodato",2158500.0
"Alaluf, Rosario",1295100.0
"Alcorta, Ricardo",1928700.0
"Algorta Brit, Felipe José",259020.0


In [12]:
print(convocatoria_df.columns)
convocatoria_df[['titular']].drop_duplicates()

Index(['camara', 'departamento', 'duracion_segundos', 'fecha_fin', 'fecha_ini',
       'id', 'id_legislador', 'lema', 'sublema', 'titular', 'nombre'],
      dtype='object')


,titular
0,NaN
4,"Ayala, Patricia"
31,"Olivera, Nicolás"
61,"Alonso, Verónica"
63,"Besozzi, Guillermo"
72,"Lacalle Pou, Luis Alberto"
75,"Delgado, Alvaro"
89,"Saravia, Jorge"
103,"Coutinho Rodríguez, Germán"
109,"Aviaga Martínez, María Carol"
